# Data preparation - Seatle AirBnB

In [55]:
#Importing standard liabraries

%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [56]:
# Uploading AirBnB data sets

calendar_df = pd.read_csv('calendar.csv')

listing_df = pd.read_csv('listings.csv')

reviews_df = pd.read_csv('reviews.csv')

As we've seen in the __Data understanding__ step: 

- Some of the attributes from Seattle dataset have missing data
    - Luckily for us only a small proportion of those attributes we are going to use in the model have around 10% of missing values. The rest of data is complete.
- We've identified highly correlated attributes from the same categories
    - Those attributes will be excluded from the training model 

It certainly does not make sense to use all categorical and numerical values for our data model. Hence we will focus on the most reasonable predictors. For instance for numerical variables we will exclude all highly  correlated attributes, this will save us from multicollinearity issues in the future. (See Data Understanding - correlation heatmap)

As for categorical, we will have to apply a bit of common sense and exclude such as: state, city, picture_url, name etc.

In [57]:
# Cleaning categorical variables. 

listings_clean_df = listing_df.drop(columns = ['amenities','calendar_updated','cancellation_policy','city',
                                               'space', 'neighborhood_overview', 'notes',
                                               'medium_url', 'xl_picture_url', 'host_url', 'is_location_exact',
                                               'country','country_code','description','experiences_offered','calendar_last_scraped',
                                               'host_has_profile_pic','host_id','host_name','host_picture_url', 'host_about', 'host_verifications',
                                               'host_thumbnail_url','listing_url','market','name','neighbourhood_group_cleansed',
                                               'jurisdiction_names', 'require_guest_profile_picture', 'smart_location',
                                               'picture_url','scrape_id','state','street','summary','thumbnail_url','zipcode'])



listings_clean_df.columns

Index(['id', 'last_scraped', 'transit', 'host_since', 'host_location',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type',
       'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'bed_type', 'square_feet', 'price', 'weekly_price', 'monthly_price',
       'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people',
       'minimum_nights', 'maximum_nights', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'number_of_reviews', 'first_review', 'last_review',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_s

In [58]:
# Cleaning categorical variables. 

listings_clean_df = listings_clean_df.drop(columns = ['id','host_response_rate','accommodates','bedrooms',
                                                      'host_listings_count','host_total_listings_count', 
                                                     'guests_included','availability_60','availability_90', 'availability_365',
                                                     'review_scores_accuracy','review_scores_cleanliness','review_scores_checkin',
                                                     'review_scores_communication','review_scores_value'])
listings_clean_df.columns

Index(['last_scraped', 'transit', 'host_since', 'host_location',
       'host_response_time', 'host_acceptance_rate', 'host_is_superhost',
       'host_neighbourhood', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type',
       'room_type', 'bathrooms', 'beds', 'bed_type', 'square_feet', 'price',
       'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee',
       'extra_people', 'minimum_nights', 'maximum_nights', 'has_availability',
       'availability_30', 'number_of_reviews', 'first_review', 'last_review',
       'review_scores_rating', 'review_scores_location', 'requires_license',
       'license', 'instant_bookable', 'require_guest_phone_verification',
       'calculated_host_listings_count', 'reviews_per_month'],
      dtype='object')

In [59]:
# Let's review attributes with more than 10% of missing values 

missing_10_pct_values = set(listings_clean_df.columns[listings_clean_df.isnull().mean() > 0.1])

missing_10_pct_values

{'cleaning_fee',
 'first_review',
 'host_acceptance_rate',
 'host_response_time',
 'last_review',
 'license',
 'monthly_price',
 'neighbourhood',
 'review_scores_location',
 'review_scores_rating',
 'reviews_per_month',
 'security_deposit',
 'square_feet',
 'transit',
 'weekly_price'}

In [60]:
# Let's drop columns with more than 10% missing values

listings_clean_df = listings_clean_df.drop(columns = ['cleaning_fee','first_review','host_acceptance_rate',
                                                     'last_review','license','monthly_price','neighbourhood',
                                                     'reviews_per_month','security_deposit','square_feet',
                                                     'transit','weekly_price'])

listings_clean_df.head(5)

,last_scraped,host_since,host_location,host_response_time,host_is_superhost,host_neighbourhood,host_identity_verified,neighbourhood_cleansed,latitude,longitude,...,maximum_nights,has_availability,availability_30,number_of_reviews,review_scores_rating,review_scores_location,requires_license,instant_bookable,require_guest_phone_verification,calculated_host_listings_count
0,2016-01-04,2011-08-11,"Seattle, Washington, United States",within a few hours,f,Queen Anne,t,West Queen Anne,47.636289,-122.371025,...,365,t,14,207,95.0,9.0,f,f,f,2
1,2016-01-04,2013-02-21,"Seattle, Washington, United States",within an hour,t,Queen Anne,t,West Queen Anne,47.639123,-122.365666,...,90,t,13,43,96.0,10.0,f,f,t,6
2,2016-01-04,2014-06-12,"Seattle, Washington, United States",within a few hours,f,Queen Anne,t,West Queen Anne,47.629724,-122.369483,...,30,t,1,20,97.0,10.0,f,f,f,2
3,2016-01-04,2013-11-06,"Seattle, Washington, United States",NaN,f,Queen Anne,t,West Queen Anne,47.638473,-122.369279,...,1125,t,0,0,NaN,NaN,f,f,f,1
4,2016-01-04,2011-11-29,"Seattle, Washington, United States",within an hour,f,Queen Anne,t,West Queen Anne,47.632918,-122.372471,...,1125,t,30,38,92.0,9.0,f,f,f,1


We can merge two columns last_scraped and host_since to calculate for how long host was on the market

In [61]:
import datetime

x = str('2015-02-24')

x = datetime.datetime.strptime(x, "%Y-%m-%d").date()

x

datetime.date(2015, 2, 24)

In [62]:
# function for converting string to datetime

def str_to_date(x):
        x = datetime.datetime.strptime(x, "%Y-%m-%d").date()
        return x

In [63]:
# converting last_scraped into datetime object

listings_clean_df['last_scraped'] = listings_clean_df['last_scraped'].apply(str_to_date)

In [64]:
# converting last_scraped into datetime object
# we fill nan for host_since with '2016-01-04' - the scrape date

listings_clean_df['host_since'] = listings_clean_df['host_since'].fillna(str('2016-01-04'))

listings_clean_df['host_since'] = listings_clean_df['host_since'].astype(str).apply(str_to_date)

In [65]:
# Calculating days with abnb

listings_clean_df['days_with_abnb'] = listings_clean_df['last_scraped'] - listings_clean_df['host_since']

listings_clean_df['days_with_abnb'] = listings_clean_df['days_with_abnb'] / np.timedelta64(1, 'D')

In [66]:
listings_clean_df.columns

Index(['last_scraped', 'host_since', 'host_location', 'host_response_time',
       'host_is_superhost', 'host_neighbourhood', 'host_identity_verified',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type',
       'room_type', 'bathrooms', 'beds', 'bed_type', 'price', 'extra_people',
       'minimum_nights', 'maximum_nights', 'has_availability',
       'availability_30', 'number_of_reviews', 'review_scores_rating',
       'review_scores_location', 'requires_license', 'instant_bookable',
       'require_guest_phone_verification', 'calculated_host_listings_count',
       'days_with_abnb'],
      dtype='object')

In [67]:
listings_clean_df.head(5)

,last_scraped,host_since,host_location,host_response_time,host_is_superhost,host_neighbourhood,host_identity_verified,neighbourhood_cleansed,latitude,longitude,...,has_availability,availability_30,number_of_reviews,review_scores_rating,review_scores_location,requires_license,instant_bookable,require_guest_phone_verification,calculated_host_listings_count,days_with_abnb
0,2016-01-04,2011-08-11,"Seattle, Washington, United States",within a few hours,f,Queen Anne,t,West Queen Anne,47.636289,-122.371025,...,t,14,207,95.0,9.0,f,f,f,2,1607.0
1,2016-01-04,2013-02-21,"Seattle, Washington, United States",within an hour,t,Queen Anne,t,West Queen Anne,47.639123,-122.365666,...,t,13,43,96.0,10.0,f,f,t,6,1047.0
2,2016-01-04,2014-06-12,"Seattle, Washington, United States",within a few hours,f,Queen Anne,t,West Queen Anne,47.629724,-122.369483,...,t,1,20,97.0,10.0,f,f,f,2,571.0
3,2016-01-04,2013-11-06,"Seattle, Washington, United States",NaN,f,Queen Anne,t,West Queen Anne,47.638473,-122.369279,...,t,0,0,NaN,NaN,f,f,f,1,789.0
4,2016-01-04,2011-11-29,"Seattle, Washington, United States",within an hour,f,Queen Anne,t,West Queen Anne,47.632918,-122.372471,...,t,30,38,92.0,9.0,f,f,f,1,1497.0


In [68]:
# Now we can drop last_scraped & 
listings_clean_df = listings_clean_df.drop(columns = ['last_scraped','host_since'])

listings_clean_df.head(5)

,host_location,host_response_time,host_is_superhost,host_neighbourhood,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,...,has_availability,availability_30,number_of_reviews,review_scores_rating,review_scores_location,requires_license,instant_bookable,require_guest_phone_verification,calculated_host_listings_count,days_with_abnb
0,"Seattle, Washington, United States",within a few hours,f,Queen Anne,t,West Queen Anne,47.636289,-122.371025,Apartment,Entire home/apt,...,t,14,207,95.0,9.0,f,f,f,2,1607.0
1,"Seattle, Washington, United States",within an hour,t,Queen Anne,t,West Queen Anne,47.639123,-122.365666,Apartment,Entire home/apt,...,t,13,43,96.0,10.0,f,f,t,6,1047.0
2,"Seattle, Washington, United States",within a few hours,f,Queen Anne,t,West Queen Anne,47.629724,-122.369483,House,Entire home/apt,...,t,1,20,97.0,10.0,f,f,f,2,571.0
3,"Seattle, Washington, United States",NaN,f,Queen Anne,t,West Queen Anne,47.638473,-122.369279,Apartment,Entire home/apt,...,t,0,0,NaN,NaN,f,f,f,1,789.0
4,"Seattle, Washington, United States",within an hour,f,Queen Anne,t,West Queen Anne,47.632918,-122.372471,House,Entire home/apt,...,t,30,38,92.0,9.0,f,f,f,1,1497.0


In [69]:
listings_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 27 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   host_location                     3810 non-null   object 
 1   host_response_time                3295 non-null   object 
 2   host_is_superhost                 3816 non-null   object 
 3   host_neighbourhood                3518 non-null   object 
 4   host_identity_verified            3816 non-null   object 
 5   neighbourhood_cleansed            3818 non-null   object 
 6   latitude                          3818 non-null   float64
 7   longitude                         3818 non-null   float64
 8   property_type                     3817 non-null   object 
 9   room_type                         3818 non-null   object 
 10  bathrooms                         3802 non-null   float64
 11  beds                              3817 non-null   float64
 12  bed_ty

### Data Cleaning

In [72]:
# Removing all rows without price

listings_clean_df = listings_clean_df.dropna(subset=['price'],axis=0)

listings_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3818 entries, 0 to 3817
Data columns (total 27 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   host_location                     3810 non-null   object 
 1   host_response_time                3295 non-null   object 
 2   host_is_superhost                 3816 non-null   object 
 3   host_neighbourhood                3518 non-null   object 
 4   host_identity_verified            3816 non-null   object 
 5   neighbourhood_cleansed            3818 non-null   object 
 6   latitude                          3818 non-null   float64
 7   longitude                         3818 non-null   float64
 8   property_type                     3817 non-null   object 
 9   room_type                         3818 non-null   object 
 10  bathrooms                         3802 non-null   float64
 11  beds                              3817 non-null   float64
 12  bed_ty

In [73]:
fill_mean = lambda col: col.fillna(col.mean())

try:
    listings_clean_df.apply(fill_mean, axis=0)
except:
    print('That broke...because column is a string.')

That broke...because column is a string.


In [74]:
listings_clean_df[['bathrooms','beds','minimum_nights','maximum_nights']].apply(fill_mean, axis=0)

,bathrooms,beds,minimum_nights,maximum_nights
0,1.0,1.0,1,365
1,1.0,1.0,2,90
2,4.5,7.0,4,30
3,1.0,2.0,1,1125
4,2.0,3.0,1,1125
...,...,...,...,...
3813,2.0,3.0,3,1125
3814,1.0,2.0,2,29
3815,1.0,1.0,1,7
3816,1.0,1.0,3,1125
